In [15]:
using Pkg
using Revise
Pkg.activate(dirname(dirname(@__DIR__)))
using Macro
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools
using JuMP

  Activating project at `~/Macro`


In [16]:
case_path = @__DIR__
println("###### ###### ######")
println("Running case at $(case_path)")

system = Macro.load_system(case_path)

model = Macro.generate_model(system)

Macro.set_optimizer(model, Gurobi.Optimizer);

Macro.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)

Macro.optimize!(model)

###### ###### ######
Running case at /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_test


┌ Info: Loading JSON data from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_test/system_data.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from /home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_test/system_data.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading JSON data from system/nodes.json
└ @ Macro /home/al3792/Macro/src/load_inputs/load_macroobject.jl:135
┌ Info: Loading CSV data from system/demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:12
┌ Info: Loading CSV data from system/demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading CSV data from system/hourly_cement_demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:12
┌ Info: Loading CSV data from system/hourly_cement_demand.csv
└ @ Macro /home/al3792/Macro/src/load_inputs/file_io/csv.jl:14
┌ Info: Loading JSON data from assets/

system.settings.Scaling = false


┌ Info: Starting model generation
└ @ Macro /home/al3792/Macro/src/generate_model.jl:3
┌ Info: Adding linking variables
└ @ Macro /home/al3792/Macro/src/generate_model.jl:17
┌ Info: Defining available capacity
└ @ Macro /home/al3792/Macro/src/generate_model.jl:20
┌ Info: Generating planning model
└ @ Macro /home/al3792/Macro/src/generate_model.jl:23
┌ Info: Generating operational model
└ @ Macro /home/al3792/Macro/src/generate_model.jl:26
┌ Info: Model generation complete, it took 12.549198150634766 seconds
└ @ Macro /home/al3792/Macro/src/generate_model.jl:31


Set parameter LicenseID to value 197246
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "license.rc.princeton.edu"
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Red Hat Enterprise Linux 8.10 (Ootpa)")

CPU model: Intel(R) Xeon(R) Gold 6246R CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 32 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 788422 rows, 525638 columns and 2688272 nonzeros
Model fingerprint: 0xaea4ed29
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+07]
Presolve removed 272117 rows and 166996 columns
Presolve time: 1.17s
Presolved: 516305 rows, 358642 columns, 2074512 

In [17]:
case = "capacity_80pct_co2_cap"

capacity_results = Macro.get_optimal_asset_capacity(system)
results_dir = joinpath(case_path, "results")
mkpath(results_dir)
Macro.write_csv(joinpath(results_dir, case * "_capacity.csv"), capacity_results)
println("wrote results")

wrote results


In [ ]:
co2_nodes = Macro.get_nodes_sametype(system.locations, CO2)

In [34]:
results_8760 = DataFrame(
    cement_plant_1 = Macro.value.(Macro.flow(system.assets[1].cement_edge)).data,
    cement_plant_2 = Macro.value.(Macro.flow(system.assets[2].cement_edge)).data,
    cement_plant_3 = Macro.value.(Macro.flow(system.assets[3].cement_edge)).data,

    cement_plant_1_emissions = Macro.value.(Macro.flow(system.assets[1].co2_edge)).data,
    cement_plant_2_emissions = Macro.value.(Macro.flow(system.assets[2].co2_edge)).data,
    cement_plant_3_emissions = Macro.value.(Macro.flow(system.assets[3].co2_edge)).data,

    ng_1_emissions = Macro.value.(Macro.flow(system.assets[1].co2_edge)).data,
    ng_2_emissions = Macro.value.(Macro.flow(system.assets[2].co2_edge)).data,
    ng_3_emissions = Macro.value.(Macro.flow(system.assets[3].co2_edge)).data,

    total_emissions = Macro.value(sum(co2_nodes[1].operation_expr[:emissions]))
)

Row,cement_plant_1,cement_plant_2,cement_plant_3,cement_plant_1_emissions,cement_plant_2_emissions,cement_plant_3_emissions,ng_1_emissions,ng_2_emissions,ng_3_emissions,total_emissions
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.385105,0.380088,150.879,0.298841,0.294948,117.082,0.298841,0.294948,117.082,1.816e7
2,0.402296,0.397062,150.844,0.312182,0.30812,117.055,0.312182,0.30812,117.055,1.816e7
3,72.1464,76.5211,2.97638,55.9856,59.3803,2.30967,55.9856,59.3803,2.30967,1.816e7
4,49.605,63.7316,38.3072,38.4935,49.4557,29.7264,38.4935,49.4557,29.7264,1.816e7
5,38.3467,52.4927,60.8044,29.7571,40.7343,47.1842,29.7571,40.7343,47.1842,1.816e7
6,39.824,55.4283,56.3916,30.9034,43.0123,43.7598,30.9034,43.0123,43.7598,1.816e7
7,39.917,55.8826,55.8443,30.9756,43.3649,43.3352,30.9756,43.3649,43.3352,1.816e7
8,39.9123,56.057,55.6746,30.9719,43.5002,43.2035,30.9719,43.5002,43.2035,1.816e7
9,39.856,56.1363,55.6515,30.9283,43.5618,43.1856,30.9283,43.5618,43.1856,1.816e7


In [35]:
Macro.write_csv(joinpath(results_dir, case * "_8760_results.csv"), results_8760)

"/home/al3792/Macro/ExampleSystems/three_zones_macro_genx_cement_test/results/capacity_80pct_co2_cap_8760_results.csv"